Merging and joining

In [8]:
import sqlite3
import pandas as pd

In [24]:
with sqlite3.connect("chinook.db") as conn:
    df_customers = pd.read_sql_query("SELECT * FROM customers", conn)
    df_invoices = pd.read_sql_query("SELECT * FROM invoices", conn)
    df_merged = pd.merge(df_customers, df_invoices,on="CustomerId", how='inner')
invoices = df_invoices['CustomerId'].value_counts().reset_index()
print(invoices)   



    CustomerId  count
0            2      7
1            5      7
2           53      7
3            6      7
4           29      7
5           30      7
6           32      7
7           44      7
8            9      7
9           11      7
10          47      7
11          49      7
12          26      7
13          28      7
14           7      7
15          50      7
16          43      7
17          45      7
18          22      7
19          24      7
20           1      7
21           3      7
22          39      7
23          41      7
24          18      7
25          20      7
26          56      7
27          58      7
28          51      7
29          27      7
30           4      7
31          21      7
32           8      7
33          14      7
34          23      7
35          37      7
36          38      7
37          40      7
38          42      7
39          46      7
40          52      7
41          16      7
42          17      7
43          19      7
44        

In [31]:
df_movie1 = pd.read_csv("movie.csv")[['director_name', 'color']]
df_movie2 = pd.read_csv("movie.csv")[['director_name', 'num_critic_for_reviews']]
left_join = pd.merge(df_movie1, df_movie2, on='director_name', how="left")
outer_join = pd.merge(df_movie1, df_movie2, on='director_name', how="outer")
print(left_join.shape[0])
print(outer_join.shape[0])


30300
30300


Grouping and Aggregating

In [33]:
df_titanic = pd.read_excel('titanic.xlsx')
avg_age = df_titanic.groupby('Pclass')['Age'].mean()
total_fare = df_titanic.groupby("Pclass")['Fare'].sum()
count = df_titanic.groupby('Pclass')['Pclass'].count()

result = pd.DataFrame({
    "Average_age":avg_age,
    "Total_Fare":total_fare,
    "Count_of Passengers":count
}).reset_index()
result

,Pclass,Average_age,Total_Fare,Count_of Passengers
0,1,38.233441,18177.4125,216
1,2,29.877630,3801.8417,184
2,3,25.140620,6714.6951,491


In [42]:
df_movie = pd.read_csv("movie.csv")
df_movie = df_movie.groupby(['color', "director_name"])[["num_critic_for_reviews","duration"]].agg({
    'num_critic_for_reviews' : 'sum',
    'duration': 'mean'
})
df_movie

num_critic_for_reviews  duration
color           director_name                                       
Black and White Akira Kurosawa                       153.0     202.0
                Aleksey German                       121.0     177.0
                Alex Garland                         489.0     108.0
                Alexander Payne                      433.0     115.0
                Alfred Hitchcock                     434.0     119.0
...                                                    ...       ...
Color           Zoran Lisinac                         17.0     108.0
                Álex de la Iglesia                    71.0     104.0
                Émile Gaudreault                      67.0      92.0
                Éric Tessier                           9.0      99.0
                Étienne Faure                          9.0      98.0

[2490 rows x 2 columns]

In [46]:
df_flights = pd.read_parquet('data/flights')
num_flights = df_flights.shape[0]
avg_arr_delay = df_flights.groupby(['year', 'months'])['ArrDelay', 'DepDelay'].agg({
    'ArrDelay' : 'mean',
    'DepDelay': 'max'
})

FileNotFoundError: [Errno 2] No such file or directory: 'data/flights'

In [50]:
df_titanic = pd.read_excel('titanic.xlsx')
def categorize(x):
    if x<18:
        return 'Child'
    return "Adult"
ct = df_titanic['Age'].apply(categorize)
df_titanic["Age_Group"] = ct
df_titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Group
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Adult
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Adult
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Adult
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Adult
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,Adult
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Adult
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,Adult
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,Adult


In [ ]:
df_employee = pd.read_csv('data/employee.csv')
df_employee['Normalized Salary'] = df_employee.groupby('DEPARTMENT')['BASE_SALARY'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)
print(df_employee)

In [ ]:
def filter_survivors(df):
    return df[df['Survived'] == 1]

def fill_missing_age(df):
    mean_age = df['Age'].mean()
    return df.assign(Age=df['Age'].fillna(mean_age))

def add_fare_per_age(df):
    df['Fare_Per_Age'] = df['Fare']/df['Age']
    return df

df_titanic_final = (
    df_titanic.pipe(filter_survivors)
    .pipe(fill_missing_age)
    .pipe(add_fare_per_age)
    )

print(df_titanic_final)

In [ ]:
def filter_delay(df):
    return df[df['DepDelay']>30]

def add_delay_per_hour(df):
    df['Delay_Per_Hour'] = df['DepDelay']/df['Duration']
    return df

df_flights_modified = (
    df_flights.pipe(filter_delay)
    .pipe(add_delay_per_hour)
    )

print(df_flights_modified)